In [5]:
import pandas as pd

df = pd.read_csv('C:\\Users\\Smile\\Downloads\\Financial Sample.csv');
print(df.head())

      Segment  Country     Product   Discount Band   Units Sold   \
0  Government   Canada   Carretera            None     1,618.50    
1  Government  Germany   Carretera            None        1,321    
2   Midmarket   France   Carretera            None        2,178    
3   Midmarket  Germany   Carretera            None          888    
4   Midmarket   Mexico   Carretera            None        2,470    

    Manufacturing Price    Sale Price   Gross Sales   Discounts     Sales   \
0                      3            20       32,370         -      32,370    
1                      3            20       26,420         -      26,420    
2                      3            15       32,670         -      32,670    
3                      3            15       13,320         -      13,320    
4                      3            15       37,050         -      37,050    

      COGS    Profit       Date  Month Number  Month Name   Year  
0   16,185    16,185   1/1/2014             1     Janua

In [13]:
import pandas as pd
import math

# Load the finance dataset into a pandas DataFrame
df = pd.read_csv('C:\\Users\\Smile\\Downloads\\Financial Sample.csv')

# Group the data by each quasi-identifier attribute and calculate the entropy
qi_entropy = {}
for col in ['Country', 'Month Number']:
    counts = df.groupby(col).size()
    probs = counts / len(df)
    qi_entropy[col] = -sum(p * math.log(p, 2) for p in probs)

# Print the entropy for each quasi-identifier attribute
for col, ent in qi_entropy.items():
    print(col, 'entropy:', ent)


Country entropy: 2.321928094887362
Month Number entropy: 3.3841837197791884


# ANONYMIZED DATASET

In [18]:
import pandas as pd
import numpy as np

# Load the dataset into a pandas dataframe
df = pd.read_csv('C:\\Users\\Smile\\Downloads\\Financial Sample.csv')

# Define the columns to use for k-anonymization
columns_to_anonymize = ['Year', 'Month Number']

# Define the value of k
k = 1
# Define a function to anonymize the values in a column
def anonymize_column(values):
    unique_values, value_counts = np.unique(values, return_counts=True)
    sorted_indices = np.argsort(value_counts)
    group_sizes = np.zeros_like(value_counts)
    group_id = np.zeros_like(values)
    for i, value_index in enumerate(sorted_indices):
        group_id[values == unique_values[value_index]] = i
        group_sizes[i] = value_counts[value_index]
    group_id[group_sizes[group_id] < k] = len(sorted_indices)
    return group_id

# Anonymize the columns in the dataframe
group_ids = []
for column in columns_to_anonymize:
    group_id = anonymize_column(df[column].values)
    group_ids.append(group_id)
df['New Month Number'] = np.max(group_ids, axis=0)
df['New Year'] = np.max(group_ids, axis=0)

# Save the anonymized dataframe to a new csv file
df.to_csv('C:\\Users\\Smile\\Downloads\\anonymized_dataset.csv', index=False)

print(df.head());

      Segment  Country     Product   Discount Band   Units Sold   \
0  Government   Canada   Carretera            None     1,618.50    
1  Government  Germany   Carretera            None        1,321    
2   Midmarket   France   Carretera            None        2,178    
3   Midmarket  Germany   Carretera            None          888    
4   Midmarket   Mexico   Carretera            None        2,470    

    Manufacturing Price    Sale Price   Gross Sales   Discounts     Sales   \
0                      3            20       32,370         -      32,370    
1                      3            20       26,420         -      26,420    
2                      3            15       32,670         -      32,670    
3                      3            15       13,320         -      13,320    
4                      3            15       37,050         -      37,050    

      COGS    Profit       Date  Month Number  Month Name   Year  \
0   16,185    16,185   1/1/2014             1     Janu

# MASKED DATASET AND ANONYMIZED DATASET

In [19]:
import pandas as pd
import numpy as np

# Load the dataset into a pandas dataframe
df = pd.read_csv('C:\\Users\\Smile\\Downloads\\anonymized_dataset.csv')

# Define the column to mask
column_to_mask = 'Country'

# Define the masking character or string
masking_char = '*'

# Mask the column by replacing the original values with asterisks
df[column_to_mask] = df[column_to_mask].apply(lambda x: masking_char * len(str(x)))

# Save the masked dataframe to a new csv file
df.to_csv('C:\\Users\\Smile\\Downloads\\masked_dataset.csv', index=False)

print(df.head())

      Segment  Country     Product   Discount Band   Units Sold   \
0  Government   ******   Carretera            None     1,618.50    
1  Government  *******   Carretera            None        1,321    
2   Midmarket   ******   Carretera            None        2,178    
3   Midmarket  *******   Carretera            None          888    
4   Midmarket   ******   Carretera            None        2,470    

    Manufacturing Price    Sale Price   Gross Sales   Discounts     Sales   \
0                      3            20       32,370         -      32,370    
1                      3            20       26,420         -      26,420    
2                      3            15       32,670         -      32,670    
3                      3            15       13,320         -      13,320    
4                      3            15       37,050         -      37,050    

      COGS    Profit       Date  Month Number  Month Name   Year  \
0   16,185    16,185   1/1/2014             1     Janu

# TIME EFFICIENCY OF K-ANONYMIZATION

In [6]:
import pandas as pd
import time

# Load the dataset into a pandas dataframe
df = pd.read_csv('C:\\Users\\Smile\\Downloads\\masked_dataset.csv')

# Define the columns to use for k-anonymization
qi_cols = ['Segment', 'Month Number', 'Year']

# Define a list of k values to try
k_values = [2, 5, 10, 20]

# Measure the time taken to perform k-anonymization for each k value
for k in k_values:
    start_time = time.time()
    
    # Group the dataset based on the QI attributes and assign a group ID to each group
    group_cols = df[qi_cols]
    group_cols['group_id'] = pd.factorize(group_cols.apply(tuple, axis=1), sort=True)[0] % k
    df['group_id'] = group_cols['group_id']
    
    end_time = time.time()
    print(f"k={k}: time={end_time-start_time}")


k=2: time=0.010561943054199219
k=5: time=0.0011260509490966797
k=10: time=0.013653993606567383
k=20: time=0.006986856460571289


C:\Users\Smile\AppData\Local\Temp/ipykernel_27532/3806932696.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  group_cols['group_id'] = pd.factorize(group_cols.apply(tuple, axis=1), sort=True)[0] % k


In [12]:
import pandas as pd
import math

# Load the finance dataset into a pandas DataFrame
df = pd.read_csv('C:\\Users\\Smile\\Downloads\\masked_dataset.csv')

# Group the data by each quasi-identifier attribute and calculate the entropy
qi_entropy = {}
for col in ['Country', 'Month Number']:
    counts = df.groupby(col).size()
    probs = counts / len(df)
    qi_entropy[col] = -sum(p * math.log(p, 2) for p in probs)

# Print the entropy for each quasi-identifier attribute
for col, ent in qi_entropy.items():
    print(col, 'entropy:', ent)


Country entropy: 1.3709505944546687
Month Number entropy: 3.3841837197791884
